### Working with Bedrock API

Example of using Converse API and tool use via Lambda for external task

In [2]:
def print_response(response):
    model_response = response.get('output', {}).get('message', {}).get('content', [{}])[0].get('text', '')

    print("✈️ Your suggested travel destinations:")
    print(model_response)

In [1]:
import json
import boto3


bedrock = boto3.client(service_name="bedrock-runtime")
bedrock_agent = boto3.client(service_name="bedrock-agent")

MODEL_ID = "amazon.nova-micro-v1:0"

In [3]:
temperature = .7

inference_config = {"temperature": temperature}

system_prompts = [{"text": "You are a virtual travel assistant that suggests destinations based on user preferences."
                + "Only return destination names and a brief description."}]

messages = []

message_1 = {
    "role": "user",
    "content": [{"text": "Create a list of 3 travel destinations."}]
}

messages.append(message_1)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
1. **Kyoto, Japan**  
   Experience traditional Japanese culture with stunning temples, beautiful gardens, and historic tea houses.

2. **Banff, Canada**  
   Enjoy breathtaking mountain scenery, outdoor activities like hiking and skiing, and charming alpine towns.

3. **Santorini, Greece**  
   Discover picturesque white-washed villages, stunning sunsets, and crystal-clear blue waters.


In [5]:
message_2 = {
        "role": "user",
        "content": [{"text": "Only suggest travel locations that are no more than one short flight away."}]
}

messages.append(message_2)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
1. **Orlando, Florida**
   - Known for its world-famous theme parks like Disney World and Universal Studios.

2. **Miami, Florida**
   - Famous for its vibrant nightlife, beautiful beaches, and rich cultural diversity.

3. **San Diego, California**
   - Renowned for its stunning beaches, world-class zoo, and beautiful coastal scenery.


In [6]:
message_3 = {
        "role": "user",
        "content": [{"text": "List only destinations by their proximity to \
        Seattle WA"}]
}

messages.append(message_3)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
1. **Vancouver, Canada**  
   A vibrant city known for its stunning natural beauty, diverse culture, and iconic landmarks like Stanley Park.

2. **Portland, Oregon**  
   Famous for its thriving food scene, lush parks, and quirky neighborhoods, offering a mix of urban and natural experiences.

3. **Victoria, British Columbia**  
   A charming city with a rich history, beautiful gardens, and the historic Royal British Columbia Museum.


In [7]:
try:
    response = bedrock_agent.create_prompt(
        name="Travel-Agent-Prompt",
        description="Checks if all trip information has been provided.",
        variants=[
            { 
                "name": "Variant1",
                "modelId": MODEL_ID,
                "templateType": "CHAT",
                "inferenceConfiguration": {
                    "text": {
                        "temperature": 0.4
                    }
                },
                "templateConfiguration": { 
                    "chat": {
                        'system': [ 
                            {
                                "text": """You are a travel agent evaluating trip requests for custom itineraries. 
                                Review the message carefully and answer YES or NO to the following screening questions. 
                                Be strict—if any detail is missing or unclear, answer NO.

                                A) Is the destination clearly stated?
                                B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
                                C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
                                D) Is there any mention of a valid passport or travel documentation?
                                E) Is there enough information to follow up with a proposed itinerary?"""
                            }
                        ],
                        'messages': [{
                            'role': 'user',
                            'content': [ 
                                {
                                    'text': "Trip request: {{event_request}}"
                                }
                            ]
                        }],
                        'inputVariables' : [
                            { 'name' : 'event_request'}
                        ]
                    }
                }
        }]
    )
    print("Created!")
    prompt_arn = response.get("arn")
except bedrock.exceptions.ConflictException as e:
    print("Already exists!")
    response = bedrock.list_prompts()
    prompt = next((prompt for prompt in response['promptSummaries'] if prompt['name'] == "TripBooker_xyz"), None)
    prompt_arn = prompt['arn']

prompt_arn

Created!


'arn:aws:bedrock:eu-west-2:035663780217:prompt/8E436JGT56'

In [8]:
response = bedrock.converse(
    modelId=prompt_arn,
    promptVariables={
        'event_request': {
            'text': """
                Hi there! I'm planning a trip to Italy with my partner and would love some help organizing the itinerary. We're hoping to travel between September 10–20 this year, ideally flying into Rome and spending a few days in Florence and Venice as well. We’d love recommendations on tours, cultural sites, and good local restaurants. We’re not interested in anything risky like skydiving or hiking remote trails — just want a relaxing and enriching experience. We both have valid passports. Let me know what other details you need!
                """
        }
    },
)
print(response['output']['message']['content'][0]['text'])

YES

A) Is the destination clearly stated?
YES. The destination is clearly stated as Italy.

B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
YES. The travel dates are within a reasonable range, September 10–20 this year.

C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
YES. The request specifies a desire for a relaxing and enriching experience, avoiding risky activities like skydiving or remote hiking.

D) Is there any mention of a valid passport or travel documentation?
YES. Both travelers have valid passports.

E) Is there enough information to follow up with a proposed itinerary?
YES. The request provides sufficient information to create a proposed itinerary, including the destination, travel dates, cities to visit (Rome, Florence, Venice), and preferences for tours, cultural sites, and local restaurants.


In [11]:
# Setup Bedrock and Lambda
import boto3
import json

bedrock = boto3.client(service_name='bedrock-runtime')
lambda_client = boto3.client("lambda", region_name="us-east-1")
MODEL_ID = "amazon.nova-micro-v1:0"

# Define the calculation tool
math_tool = {
    "toolSpec": {
        "name": "calculateNumbers",
        "description": "Performs basic arithmetic operations",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "operation": {"type": "string"},
                    "num1": {"type": "number"},
                    "num2": {"type": "number"}
                },
                "required": ["operation", "num1", "num2"]
            }
        }
    }
}

# Function to trigger the Lambda calculation service
def execute_calculation(input_data):
    response = lambda_client.invoke(
        FunctionName="math-function",  
        InvocationType="RequestResponse",
        Payload=json.dumps(input_data)
    )
    response_payload = response["Payload"].read()
    calculation_result = json.loads(response_payload)
    response_body = calculation_result.get("body", "{}")
    return json.loads(response_body) if isinstance(response_body, str) else response_body

# User's initial message
user_input = {
    "role": "user",
    "content": [{"text": "Please subtract 60 from 100"}]
}

# Define system instructions
system_instructions = [
    {"text": """
    You are a virtual assistant capable of performing basic arithmetic operations: add, subtract, multiply, and divide.
    If the user doesn't specify an operation, ask them for more details.
    """}
]

# First interaction with the model
first_interaction = bedrock.converse(
    modelId=MODEL_ID,
    system=system_instructions,
    messages=[user_input],
    toolConfig={
        "tools": [math_tool],
        "toolChoice": {"auto": {}}
    },
    inferenceConfig={"temperature": 0.7}
)

# Process the assistant's response to check if tool is required
assistant_reply = first_interaction["output"]["message"]
message_parts = assistant_reply["content"]
tool_request_block = next((part for part in message_parts if "toolUse" in part), None)

if not tool_request_block:
    print("=== Assistant's Direct Response ===")
    print(message_parts[0]["text"])
else:
    tool_request = tool_request_block["toolUse"]
    tool_input_data = tool_request["input"]
    tool_id = tool_request["toolUseId"]
    print(tool_request_block)
    print(f"→ Assistant triggered tool: calculateNumbers with input: {tool_input_data}")

    # Execute the requested tool
    tool_result = execute_calculation(tool_input_data)
    print(f"← Lambda Function output: {tool_result}")

    # Create a response based on the tool's output
    try:
        result_summary = f"The outcome of the calculation is {tool_result['result']}."
    except Exception as e:
        result_summary = f"Oops! There was an error with the calculation. ({str(e)})"

    # Generate tool result message
    tool_response_msg = {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": tool_id,
                    "content": [{"text": result_summary}]
                }
            }
        ]
    }

    # Send tool result back to the model
    final_output = bedrock.converse(
        modelId=MODEL_ID,
        messages=[user_input, assistant_reply, tool_response_msg],
        toolConfig={  
            "tools": [math_tool],
            "toolChoice": {"auto": {}}
        },
        inferenceConfig={"temperature": 0.7}
    )

    # Display the final response from the assistant
    final_message = final_output["output"]["message"]["content"][0]["text"]
    print("\n=== Final Assistant Response ===")
    print(final_message)

{'toolUse': {'toolUseId': 'tooluse_0vrE7ZVK4VaKE2elHXAF3u', 'name': 'calculateNumbers', 'input': {'num1': 100, 'operation': 'subtract', 'num2': 60}}}
→ Assistant triggered tool: calculateNumbers with input: {'num1': 100, 'operation': 'subtract', 'num2': 60}
← Lambda Function output: {'result': 40.0}

=== Final Assistant Response ===
The outcome of subtracting 60 from 100 is 40.0.
